# CS5228 Assignment 3 - Recommender Systems & Graph Mining (50 Points)

Hello everyone, this assignment notebook covers the topics of Recommender Systems and Graph Mining. There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e. your lines of code) between sentences that "your code starts here" and "your code ends here". The space between these two lines does not reflect the required or expected lines of code :). For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed)

**Important:**
* Remember to save this Jupyter notebook as A3_YourNameInLumiNUS_YourNUSNETID.ipynb
* Submission deadline is Nov 7, 11.59 pm

-----------------------------------------------------

Please add your nusnet and student id also in the code cell below.

In [1]:
student_id = 'A0236597M'
nusnet_id = 'e0744016'

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well, but you can use the overview below to double-check that you covered everything.

* **1 Recommender Systems (30 Points)**
    * 1.1 User-Based Collaborative Filtering (6 Points)
        * 1.1 a) (4 Points)
        * 1.2 b) (2 Points)
    * 1.2 Implementing Matrix Factorization (14 Points)
        * 1.2 a) (2 Points)
        * 1.2 b) (2 Points)
        * 1.2 c) (8 Points)
        * 1.2 d) (2 Points)
    * 1.3 Questions about Recommender Systems (10 Points)
        * 1.3 a) (2 Points)
        * 1.3 b) (2 Points)
        * 1.3 c) (3 Points)
        * 1.3 d) (3 Points)    
* **2 Graph Mining - Centrality Measures (20 Points)**
    * 2.1 Implementing PageRank (12 Points)
        * 2.1 a) (2 Points)
        * 2.1 b) (8 Points)
        * 2.1 c) (2 Points)
    * 2.2 Comparing Centrality Measures (8 Points)
        * 2.2 a) (3 Points)
        * 2.3 b) (5 Points)


### Setting up the Notebook

In [2]:
%matplotlib notebook

In [3]:
import numpy as np
import pandas as pd
import networkx as nx

## 1 Recommender Systems (30 Points)

### 1.1 User-Based Collaborative Filtering (6 Points)

Throughout this task, you are free to implement your calculations or using pen and paper, but please make sure that you provide sufficient details in your submitted answer that makes it clear how you got your results. The individual tasks specify this in more detail.

Given to you is a simple rating dataset containing 6 users $u_1, u_2, \dots, u_6$, and 8 movies $m_1, m_2, \dots, m_8$, and the rating matrix $R$:

$$
R = 
\begin{bmatrix} 
    5 & 0 & \mathbf{\color{red} ?} & 2 & 0 & 5 & 4 & 0 \\
    3 & 0 & 4 & 0 & 0 & 2 & 0 & 3 \\
    5 & 0 & 3 & 3 & 0 & 0 & 3 & 0 \\
    5 & 0 & 0 & 3 & 0 & 4 & 5 & 0 \\
    0 & 3 & 2 & 0 & 0 & 1 & 0 & 3 \\
    5 & 3 & 1 & 3 & 0 & 5 & 4 & 0
\end{bmatrix}
$$

In this example, the range of the ratings is from 1 to 5.

Your task is to find the best estimate for rating $R_{u_1,m_3}$ of user $u_1$ for movie $m_3$, indicated by the red question mark in rating matrix $R$.

**1.2 a) Calculate all cosine similarities between user $u_1$ and all other users! (4 Points)!** Please complete the list of equations in the markdown below; use a precision of 2 decimals.

**Important:** Show at least for one equation how you calculate the similarity in detail. You can use an additional code or markdown cell.

**Your Answer:**

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
M = np.array([[5,0,0,2,0,5,4,0],
              [3,0,4,0,0,2,0,3],
              [5,0,3,3,0,0,3,0],
              [5,0,0,3,0,4,5,0],
              [0,3,2,0,0,1,0,3],
              [5,3,1,3,0,5,4,0]])

masked = np.ma.masked_equal(M, 0)
user_mean_ratings = np.mean(masked, axis=1).reshape(-1, 1)

M_normalized = M - user_mean_ratings
M_normalized[M == 0] = 0
user_similarities = cosine_similarity(M_normalized, M_normalized)
print('sim(u1,u2): {:.2f}'.format(user_similarities[0,1]))
print('sim(u1,u3): {:.2f}'.format(user_similarities[0,2]))
print('sim(u1,u4): {:.2f}'.format(user_similarities[0,3]))
print('sim(u1,u5): {:.2f}'.format(user_similarities[0,4]))
print('sim(u1,u6): {:.2f}'.format(user_similarities[0,5]))

sim(u1,u2): -0.29
sim(u1,u3): 0.59
sim(u1,u4): 0.74
sim(u1,u5): -0.31
sim(u1,u6): 0.48


* sim($u_1$, $u_2$) = <font color='red'>-0.29</font>
* sim($u_1$, $u_3$) = <font color='red'>0.59</font>
* sim($u_1$, $u_4$) = <font color='red'>0.74</font>
* sim($u_1$, $u_5$) = <font color='red'>-0.31</font>
* sim($u_1$, $u_6$) = <font color='red'>0.48</font>

**1.2 b) Calculate the estimated rating $R_{u_1,m_3}$ (2 Points)!** Consider the 2 most similar users for this calculation; use a precision of 2 decimals. Show how you arrived at this result! You can use an additional code or markdown cell. 

**Your Answer:**

In [5]:
user_idx = 0
movie_idx = 2
k = 2
neighbors = np.argsort(user_similarities[user_idx])[::-1]
neighbors = [ n_idx for i, n_idx in enumerate(neighbors) if M[n_idx][movie_idx] != 0 and user_similarities[user_idx][n_idx] > 0]
topk_neighbor_indices = neighbors[:k]
neighbor_similarities = user_similarities[user_idx][topk_neighbor_indices]
neighbor_similarities
neighbor_ratings = M[topk_neighbor_indices][:,movie_idx]
if np.sum(neighbor_similarities) == 0:
    user_rating = 0.0
user_rating = np.round(np.sum(neighbor_similarities * neighbor_ratings) / np.sum(neighbor_similarities), 2)
print('user 1 rated movie 3 with: {}'.format(user_rating))

user 1 rated movie 3 with: 2.1


* $R_{u_1,m_3}$ = <font color='red'>2.1</font>

### 1.2 Implementing Matrix Factorization

Matrix Factorization -- and here more specifically: non-negative Matrix Factorization -- is a class of algorithms where a matrix $M$ is factorized into (usually) two matrices $W$ and $H$, with the property that all three matrices have no negative elements. Matrix Factorization is popular techniques applied in recommender systems, where $W$ and $H$ contain a latent representation of all users and all items, respectively, and $M$ represents the rating matrix.

In this task, you will implement (non-negative) Matrix Factorization from scratch using Gradient Descent as covered in the lecture. In fact, we use the rating matrix $M$ which was used as an example in the lecture:

In [6]:
M = np.array([
    [4, 0, 0, 5, 1, 0, 0],
    [5, 5, 4, 0, 0, 0, 0],
    [0, 0, 0, 2, 4, 5, 0],
    [0, 3, 0, 0, 0, 0, 3]
], dtype=np.float)

print(M)

[[4. 0. 0. 5. 1. 0. 0.]
 [5. 5. 4. 0. 0. 0. 0.]
 [0. 0. 0. 2. 4. 5. 0.]
 [0. 3. 0. 0. 0. 0. 3.]]


/Users/anna/miniconda3/envs/deeplearn_course/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


**2.1 a) Implement method `mf_init()` that initializes matrices `W` and `H`, as well as matrix `Z`! (2 Points).** Matrix `Z` is a auxiliary matrix containing the indices of all non-zero entries of Matrix M (Hint: Have look at [`np.argwhere()`](https://numpy.org/doc/stable/reference/generated/numpy.argwhere.html) for this; below shows the expected shape and content of `Z`).

In [7]:
def mf_init(M, k=100):
    # k is the size of the latent representation
    
    Z, W, H = None, None, None
    
    num_users, num_items = M.shape

    #########################################################################################
    ### Your code starts here ###############################################################
    W = np.random.rand(num_users, k)
    H = np.random.rand(k, num_items)
    Z = np.argwhere(M != 0)
    
    ### Your code ends here #################################################################
    #########################################################################################  
    
    return Z, W, H

You can test your implementation with the code cell below. The shapes of `W` and `H` should reflect the number of users and items, as well as the size of the latent representations. The shape of `Z` should be `(num_nonzero, 2)`. For example matrix `M`, the shape should be `(11, 2)` since `M` has 11 non-zero entries.

In [8]:
Z, W, H = mf_init(M)

print('W.shape = {}'.format(W.shape))
print('H.shape = {}'.format(H.shape))
print('Z.shape = {}'.format(Z.shape))
print()
print('Z containing all the indices of all non-zero entries in M')
print(Z)

W.shape = (4, 100)
H.shape = (100, 7)
Z.shape = (11, 2)

Z containing all the indices of all non-zero entries in M
[[0 0]
 [0 3]
 [0 4]
 [1 0]
 [1 1]
 [1 2]
 [2 3]
 [2 4]
 [2 5]
 [3 1]
 [3 6]]


The output for `Z` should look like this:
```
[[0 0]
 [0 3]
 [0 4]
 [1 0]
 [1 1]
 ...]
```
indicating the `M[0, 0]`, `M[0, 1]`, `M[0, 4]`, `M[1, 0]`, `M[1, 1]`, ... are all non-zero entries.

**1.2 b) Implement method `mf_loss()` to calculate the loss w.r.t. the current values of matrices `W` and `H`! (2 Points).** 

In [9]:
def mf_loss(M, Z, W, H):
    loss = np.inf
    
    #########################################################################################
    ### Your code starts here ###############################################################    
    loss = np.sum(np.square(M-np.matmul(W,H))[M!=0])/len(Z)
    
    ### Your code ends here #################################################################
    #########################################################################################    

    return loss
        

You can test your implementation with the code cell below. We set `np.random.seed(0)` to ensure the same results in case of multiple executions.

In [10]:
np.random.seed(0)

Z, W, H = mf_init(M)

loss = mf_loss(M, Z, W, H)

print('Initial loss: {:.1f}'.format(loss))

Initial loss: 443.6


The resulting loss should be: **443.6**

**1.2 c) Complete the code cell below to implement matrix factorization using Gradient Descent! (8 Points).** The complete algorithm together with the required gradients is available as pseudo code in the lecture slides, and you are already familiar with the basic concept of Gradient Descent.

In [46]:
# Set hyperparameters
k, learning_rate, lambda_reg, num_iter = 100, 0.0001, 0.1, 100

# We first have to initialize
np.random.seed(0)
Z, W, H = mf_init(M, k=k)

for it in range(num_iter):

    #########################################################################################
    ### Your code starts here ############################################################### 
    
    # W = W + 2 * learning_rate*(np.matmul(M-np.matmul(W,H),H.transpose())+lambda_reg*W)
    # H = H + 2 * learning_rate*(np.matmul(W.transpose(), M-np.matmul(W,H))+lambda_reg*H)
    for i, j in Z:
        u = W[i]
        v = H[:,j]
        grad_u = 2 * v * (M[i,j] - np.dot(u, v)) - 2 * lambda_reg * u
        grad_v = 2 * u * (M[i,j] - np.dot(u, v)) - 2 * lambda_reg * v
        W[i] += learning_rate * grad_u
        H[:,j] += + learning_rate * grad_v

    ### Your code ends here #################################################################
    #########################################################################################           

    # Print loss every 10% of the iterations
    if(it % (num_iter/10) == 0):
        print('Loss: {:.5f} \t {:.0f}%'.format(mf_loss(M, Z, W, H), (it / (num_iter/100))))
        
# Print final loss        
print('Loss: {:.5f} \t 100%'.format(mf_loss(M, Z, W, H)))        

Loss: 421.70195 	 0%
Loss: 264.70308 	 10%
Loss: 175.59000 	 20%
Loss: 120.98281 	 30%
Loss: 85.68139 	 40%
Loss: 61.95274 	 50%
Loss: 45.52764 	 60%
Loss: 33.89706 	 70%
Loss: 25.51235 	 80%
Loss: 19.37950 	 90%
Loss: 15.23625 	 100%


With the given hyperparameter values  (`k, learning_rate, lambda_reg, num_iter = 100, 0.0001, 0.1, 100`), you should see a loss around **15.2** at the end of the training.

With our learned estimates for `W` and `H`, we can simply calculate matrix `P` as the product of `W` and `H`, representing the matrix of predicted ratings.

In [47]:
P = np.dot(W, H)

Let's see how it looks like...

In [48]:
print(np.around(P, 1))

[[ 7.  10.2 12.   7.9  5.6 10.6 12.5]
 [ 7.7  6.9  8.  11.2  9.1 14.9 13.1]
 [ 9.7  9.  10.4  7.   6.8  8.3 10.8]
 [ 9.1  7.2 10.7 11.7  9.1 12.4  9.3]]


With the given hyperparameter values (`k, learning_rate, lambda_reg, num_iter = 100, 0.0001, 0.1, 100`) the result should look something like this:

```
[[ 7.  10.2 12.   7.9  5.6 10.7 12.6]
 [ 7.8  6.9  8.1 11.3  9.1 15.  13.2]
 [ 9.7  9.  10.4  7.   6.8  8.4 10.8]
 [ 9.2  7.3 10.7 11.7  9.1 12.5  9.3]]
```

**1.2 d) Explore different hyperparameter settings and briefly explain your observations! (2 Points)** You can simply re-run all code cells from 1.2 c) onwards, just with different parameter values for `k`, `learning_rate`, `lambda_reg`, and `num_iter`.

**Your Answer:**

1. k: k is the demension of the latent representation. With larger k, the system can caputure more information of the feature which means the larger the better, but usually 100 or 150 is enough.
2. learning_rate: too large learning_rate may cause the loss to be `nan`.
3. lambda_reg: increase the value of lambda_reg, we get worse fit of known ratings, "smoother" values for all ratings. decrease the value of lambda_reg, we get better fit of known ratings, more "extreme" values of unknown ratings.
4. num_iter: large number of iteration may converge the loss to nearly 0, and the predition of known ratings are almost the same as the ground-truth ratings. The variance of the whole ratings are getting smaller with larger number of iterations. Which means the "smoother" values for all ratings.

### 1.3 Questions about Recommender Systems (10 Points)

In 1.1, the movie $m_5$ is new has not received any rating yet, so $R_{u,m_5} = 0$ for any user $u$. Thus, using basic Collaborative Filtering, $m_5$ will never be recommended and therefore is less likely to get more ratings. 

**1.3 a) For real-world recommender systems, what are common strategies to mitigate this problem? (2 Points)** Briefly sketch some possible ideas.

**Your answer:**

This is the cold-start problem for a new movie. We have no ratings of the new movie. Here are two ways to solve this problem:
1. Use content-based model at first, we compute the similarity of new movies with other movies. When we get enough ratings of movies, we can change to collaborative filtering.
2. An navie approach is to build a regression model for users with the independent variable of movie features, then use this model to predict ratings of new movies.

You get hired to build a recommendation system for a music streaming service. The platform already has 500,000 users and is giving them access to 1,000,000 songs. The platform recently introduced a feature where users can rate songs, but so far, only 10,000 ratings have been made.

**1.3 b) In line with our classification used in the lecture, what type of recommender system is best suited for the current platform? (2 Points)** Briefly explain/justify your decision.

**Your Answer:**

I would like to apply a hybrid content-based filtering recommender. There are only a few interactions, a pure collaborative algorithm can recommend it, but the quality of those recommendations will be poor, and it may not recommend unpopular songs at all. A hybrid approach can make the system robust because it considers the aspects of songs and users. In a hybrid system, characteristics, either of the songs or of the users, are weighted depending on the user’s sense of significance. Different criteria (such as the singers, languages, lyrics, and countries) will be weighted differently in order to select a song that the user would like. 

Assume you want to build movie recommendation platform that aggregates ratings of movies across different websites (e.g., [IMDb](https://www.imdb.com/), [Rotten Tomatoes](https://www.rottentomatoes.com/)). Basically all websites have their own way to quantify the movie ratings. Your goal is to build a recommender system using Collaborative Filtering.

**1.3 c) Can you combine different rating datasets to make better recommendations, and if so, what are the requirements? (3 Points)**

**Your answer:**


Yes, they can be combined to make a better prediction as long as we make proper use of the data from the two websites.
Both IMBd and Rottten Tomatoes have the same movies, but they have different evaluation systems for movies. RT is a 5-point system and imdb is a 10-point system. Therefore, the data must be unified first. While the two systems have different users, we can classify users and map the data on both sides according to user characteristics. 

In 1.2 you've implemented a model-based recommender system using (non-negative) Matrix Factorization. Since we used only a toy rating matrix, performance was not an issue here. In real-world recommendations with many users and items, Matrix Factorization can be quite time consuming. The problem is that online platforms are very dynamic: users are joining and leaving, new items are added, users add new or update previous ratings. All of those cases change the rating matrix.

**1.3 d) How do different cases (e.g., new user/item/rating) affect a current result of a Matrix Factorization for a recommender system? (3 Points)** Outline the different problems, and discuss meaningful approaches to mitigate them. For example, a new user or item refers to the *Cold-Start Problem*. What are good strategies to address the Cold-Start Problem using Matrix Factorization?

(Note: When you're discussing challenges regarding runtime/performance, please **exclude** any solutions relying on bigger clusters and parallel computing :). While those are valid points, in principle, here we want to focus on conceptual solutions).

**Your Answer:**

1. User cold start problem: A basic MF model can’t predict ratings for new users. Use Hybrid MF-Decision Tree Recommender System to solve the problem. Uses MF to find best queries to estimate user profiles.
2. Item cold start problem: How to recommend new unrated items in MF framework? Use Local Collective Embeddings to solve the problem. Uses collective factorization to link user ratings to item text.

-------------------------------------------------------------------------

## 2 Graph Mining

For your last task, we look into the concept of centrality measures which quantify the importance of nodes in a graph/network.

### Data Preparation

As graph dataset, we use the MRT network in Singapore collected from the [LTA DataMall](https://datamall.lta.gov.sg/content/dam/datamall/datasets/Geospatial/TrainStation.zip); note that we exclude LRT stations. The data comes in goe-spatial data format with several additional metadata. However, we already prepared the data and converted it into simple CSV files.

In the following, we load and prepare the data for the task. You can just run through the following code cells to get everything set up.


#### Load MRT Stations

In [14]:
df_mrt_stations = pd.read_csv('data/a3-sg-mrt-stations.csv')

df_mrt_stations.head()

,name,codes,stn_id
0,Admiralty,NS10,ba
1,Aljunied,EW9,if
2,Ang Mo Kio,NS16,vs
3,Bakau,SE3,cl
4,Bangkit,BP9,id


#### Load MRT Connections

The data of the train connections contains information about each direct connection between 2 MRT stations. Each MRT stations is identified by its unique name. 

In [15]:
df_mrt_connections = pd.read_csv('data/a3-sg-mrt-connections.csv')

df_mrt_connections.head()

,source,destination
0,novena,newton
1,pasir ris,tampines
2,sengkang,punggol
3,woodlands,marsiling
4,gul circle,tuas crescent


#### Generate Graph

From the connection data, we can now generate the train network graph where the nodes are MRT stations and there is a directed edge between two nodes if there is a direct connection between the corresponding MRT stations. While the travel time between two MRT stations would make a meaningful edge weight, we ignore it in this notebook for simplicity and assume an unweighted graph.

In [16]:
## Create an "empty" directed graph G
G = nx.DiGraph()

for idx, row in df_mrt_connections.iterrows():
    G.add_edge(row['source'], row['destination'])

#### Basic Information about the Graph

In [17]:
print('The graph has {} nodes and {} edges'.format(len(G.nodes), len(G.edges)))

The graph has 157 nodes and 354 edges


#### Create Adjacency Matrix for Graph G

We later also need the adjacency matrix of the MRT network graph. We can simple use [`to_numpy_matrix()`](https://networkx.org/documentation/networkx-1.7/reference/generated/networkx.convert.to_numpy_matrix.html) which returns the graph adjacency matrix as a numpy matrix. As the identifier of nodes are the names of the MRT stations but the matrix can only be accessed via its row and column indices, we also need a dictionary that maps the indices back to the station names.

In [18]:
## Convert graph of train connection to adjacency matrix
A_mrt = nx.to_numpy_matrix(G)

## Create a dictionary to map a matrix index to the MRT station name (which are the node identifiers)
node_map = { idx:node for idx, node in enumerate(G.nodes)  }

## Print an example
print(node_map[0])

novena


### 2.1 Implementing PageRank (14 Points)

In this task, you will implement the basic PageRank algorithm using the Power Iteration methods as introduced in the lecture.

$$
c_{PR} = \alpha M c_{PR} + (1-\alpha)E
$$

where $E = (1/n, 1/n, ..., 1/n)^T$ with $n$ being the number of nodes.

For testing and debugging your implementation, define a very basic graph by means of its adjacency matrix. In fact, this toy graph is the one used in the [original PageRank paper](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) (Page 5) from 1999.

In [19]:
A_demo = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [1, 0, 0]    
], dtype=np.float)

/Users/anna/miniconda3/envs/deeplearn_course/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


**2.1 a) Implement method `create_transition_matrix()` that converts an adjacency matrix to a transition matrix! (2 Points).** (Hint: You can find an example for this transformation in the lecture slides.)

In [20]:
def create_transition_matrix(A):
    
    M = None
    
    #########################################################################################
    ### Your code starts here ###############################################################    
    M = A.T

    M = M / M.sum(axis=0)

    M = np.nan_to_num(M)

    ### Your code ends here #################################################################
    #########################################################################################
    
    return np.asarray(M)


create_transition_matrix(A_demo)

array([[0. , 0. , 1. ],
       [0.5, 0. , 0. ],
       [0.5, 1. , 0. ]])

**2.1 b) Implement method `pagerank()` to compute the PageRank scores given an adjacency matrix A (8 Points).** (Hint: You can find an example for this transformation in the lecture slides.)

In [21]:
def pagerank(A, alpha=0.85, eps=0.0001, max_iter=1000, verbose=True):
    
    ## Generate transition matrix from adjacency matrix A
    M = create_transition_matrix(A)
    E, c = None, None
    
    #########################################################################################
    ### Your code starts here ############################################################### 

    ## Initialize E and c
    num_nodes = A.shape[0]
    E = np.ones(num_nodes)/num_nodes
    c = alpha * np.matmul(M, E) + (1-alpha) * E
    
    ### Your code ends here #################################################################
    ######################################################################################### 

    # Run the power method: iterate until differences between steps converges
    num_iter = 0
    while True:
        
        num_iter += 1

        #########################################################################################
        ### Your code starts here ###############################################################  
        old_c = c
        c = alpha * np.matmul(M, c) + (1-alpha) * E

        if (np.abs(c-old_c) <= eps).all():
            break
            
        ### Your code ends here #################################################################
        #########################################################################################            
            
        
    if verbose == True:
        print('The power method took {} iterations.'.format(num_iter))

    ## Return the results as a dictiory with the nodes as keys and the PageRank score as values
    return { k:score for k, score in enumerate(c.squeeze()) }


Test your implementation on the toy graph.

In [22]:
pagerank(A_demo, alpha=1.0)

The power method took 23 iterations.


{0: 0.3999837239583333, 1: 0.20003255208333331, 2: 0.3999837239583333}

With $\alpha=1.0$ the scores should look as follows -- apart from floating point precision issues:

`{0: 0.4, 1: 0.2, 2: 0.4}`

Again, you can check with the [original PageRank paper](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) (Page 5).

**2.1 c) Run your PageRank implementation on the MRT train network graph (2 Points)**. Find the top-5 MRT stations with respect to their PageRank scores and complete the table below. Use the code cell below to show how you get to the results; use the default value $\alpha = 0.85$. (Hint: You will need `node_map` name get the name of the MRT stations).

**Your Answer:**


| Rank | Score | MRT Station |
| ---  | ---   | ---                  |
| 1    | <font color='red'>0.01499</font> | <font color='red'>sengkang</font> |
| 2    | <font color='red'>0.01315</font> | <font color='red'>punggol</font> |
| 3    | <font color='red'>0.01126</font> | <font color='red'>dhoby ghaut</font> |
| 4    | <font color='red'>0.01116</font> | <font color='red'>tampines</font> |
| 5    | <font color='red'>0.01083</font> | <font color='red'>bukit panjang</font>  |

(for the scores, please use a precision of 5 decimals)

In [23]:
#########################################################################################
### Your code starts here ###############################################################
mrt_scores = pagerank(A_mrt, alpha=0.85)
top_5 = sorted(mrt_scores.items(),key = lambda x:x[1],reverse = True)[0:5]
top_5_score = [round(x[1],5) for x in top_5]
top_5_id = [x[0] for x in top_5]
top_5_name = [node_map[id] for id in top_5_id]
print('top 5 score:', top_5_score)
print('top 5 name:', top_5_name)
### Your code ends here #################################################################
#########################################################################################  

The power method took 17 iterations.
top 5 score: [0.01499, 0.01315, 0.01126, 0.01116, 0.01083]
top 5 name: ['sengkang', 'punggol', 'dhoby ghaut', 'tampines', 'bukit panjang']


### 2.2 Comparing Centrality Measures (8 Points)

We saw in the lecture that different centrality measures look at different topological features of a graph to quantify the importance of nodes. This task compares different measures, using the following implementations provided by `networkX`:

* [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)
* [nx.centrality.in_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.in_degree_centrality.html#networkx.algorithms.centrality.in_degree_centrality)
* [nx.centrality.out_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.out_degree_centrality.html#networkx.algorithms.centrality.out_degree_centrality)
* [nx.centrality.closeness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality)
* [nx.centrality.betweenness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html)

(**Note:** [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) might give a (slightly) different ranking than your own implementation of PageRank since this implementation is a bit modified. So don't take this as a 1:1 reference to check your PageRank implementation.)

**2.2 a) Run the 5 centrality measures on the MRT train network graph (3 Points)**. Find the top-5 MRT stations with respect to their centrality scores and complete the table below. You only need to add the name of the MRT stations, not the scores.

Use the code cell below to show how you get to the results; use the default values for all 5 implementations of the centrality measures.

**Your Answer:**


| Rank | PageRank | InDegree | OutDegree | Closeness | Betweenness |
| ---  | ---      | ---      | ---       | ---         |  --- |
| 1    |  <font color='red'>sengkang</font> | <font color='red'>sengkang</font>  | <font color='red'>sengkang</font>  | <font color='red'>serangoon</font> |  <font color='red'>serangoon</font>  |
| 2    |  <font color='red'>punggol</font>  | <font color='red'>punggol</font>  | <font color='red'>punggol</font> | <font color='red'>bishan</font>  | <font color='red'>botanic gardens</font>  |
| 3    | <font color='red'>dhoby ghaut</font>  | <font color='red'>dhoby ghaut</font>   | <font color='red'>dhoby ghaut</font> | <font color='red'>botanic gardens</font>  | <font color='red'>kovan</font>  |
| 4    | <font color='red'>tampines</font>  | <font color='red'>newton</font> |  <font color='red'>newton</font> |  <font color='red'>little india</font> | <font color='red'>bishan</font>  |
| 5    | <font color='red'>bukit panjang</font>  | <font color='red'>tampines</font> | <font color='red'>tampines</font>  |  <font color='red'>lorong chuan</font> | <font color='red'>sengkang</font>  |

In [24]:
#########################################################################################
### Your code starts here ###############################################################
page_rank = sorted(nx.pagerank(G, alpha=0.85).items(),key = lambda x:x[1],reverse = True)[0:5]
in_degree = sorted(nx.in_degree_centrality(G).items(),key = lambda x:x[1],reverse = True)[0:5]
out_degree = sorted(nx.out_degree_centrality(G).items(),key = lambda x:x[1],reverse = True)[0:5]
closeness = sorted(nx.closeness_centrality(G).items(),key = lambda x:x[1],reverse = True)[0:5]
betweenness = sorted(nx.betweenness_centrality(G).items(),key = lambda x:x[1],reverse = True)[0:5]
### Your code ends here #################################################################
#########################################################################################  

**2.2 b) Discuss the results and your observations (5 Points).** Based on the definitions and intuitions behind these 5 different centrality measures, discuss the results of 2.2 a): For each centrality, briefly describe what it means for a MRT station to have the highest score!

**Your Answer:**

1. PageRank: PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links. In the case of MRT, it returns the station which intersect most with other stations. There are three lines coming into Sengkang MRT.
2. InDegree: The in-degree centrality for a node v is the fraction of nodes its incoming edges are connected to. In the case of MRT, it returns the station which intersect most with other stations. 
3. OutDegree: The out-degree centrality for a node v is the fraction of nodes its outgoing edges are connected to. In the case of MRT, it is similar to the results of InDegree algorithm.
4. Closeness: Closeness centrality of a node u is the reciprocal of the sum of the shortest path distances from u to all n-1 other nodes. In the case of MRT, it returns the station which is located in the center of Singapore. Since the top five MRT stations are all on the circle line or in the circle line.
5. Betweenness: Betweenness centrality of a node v is the sum of the fraction of all-pairs shortest paths that pass through v. In the case of MRT, it is similar to the Closeness algorithm. The MRT in the center of Singapore tend to have higher score.
